In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from sklearn.metrics import f1_score, roc_curve, auc
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.regularizers import l2

# Load and preprocess the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Convert to float32 and normalize pixel values to [0, 1]
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

# Resize images to 64x64
train_images_resized = tf.image.resize(train_images[..., tf.newaxis], (32, 32))
test_images_resized = tf.image.resize(test_images[..., tf.newaxis], (32, 32))

# Convert to RGB channels by duplicating the single channel
train_images_rgb = tf.concat([train_images_resized] * 3, axis=-1)
test_images_rgb = tf.concat([test_images_resized] * 3, axis=-1)

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Function to calculate F1 score
def calculate_f1_score(model, test_data, test_labels):
    test_pred_probs = model.predict(test_data)
    test_pred_labels = np.argmax(test_pred_probs, axis=1)
    test_true_labels = np.argmax(test_labels, axis=1)
    f1 = f1_score(test_true_labels, test_pred_labels, average='weighted')
    return f1

# Function to plot ROC curve and AUC
def plot_roc_auc(model, test_data, test_labels):
    test_pred_probs = model.predict(test_data)
    
    plt.figure(figsize=(12, 8))
    for i in range(10):
        fpr, tpr, _ = roc_curve(test_labels[:, i], test_pred_probs[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'Class {i} (AUC = {roc_auc:.2f})')

    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.grid(True)
    plt.show()

# Function to create deeper ResNet-50-like model
def create_deeper_resnet50_model(learning_rate):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    predictions = Dense(10, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Define the learning rate
resnet_learning_rate = 0.0001

# Create and compile the deeper ResNet-50-based model
deeper_resnet50_model = create_deeper_resnet50_model(resnet_learning_rate)

# Training 
history_deeper_resnet50 = deeper_resnet50_model.fit(
    train_images_rgb, train_labels,
    epochs=5,  # Increase the number of epochs
    batch_size=512,
    validation_data=(test_images_rgb, test_labels),
    callbacks=[
        EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    ]
)

# Evaluate and print test accuracy
deeper_resnet50_test_loss, deeper_resnet50_test_acc = deeper_resnet50_model.evaluate(test_images_rgb, test_labels)
print(f"Deeper ResNet-50 Test accuracy (LR={resnet_learning_rate}): {deeper_resnet50_test_acc:.4f}")

# Calculate and print F1 score
deeper_resnet50_test_f1 = calculate_f1_score(deeper_resnet50_model, test_images_rgb, test_labels)
print(f"Deeper ResNet-50 Test F1 Score: {deeper_resnet50_test_f1:.4f}")

# Plot ROC curve and AUC
plot_roc_auc(deeper_resnet50_model, test_images_rgb, test_labels)

# Plotting training vs validation loss
plt.figure(figsize=(10, 6))
plt.plot(history_deeper_resnet50.history['loss'], label='Training Loss', color='blue')
plt.plot(history_deeper_resnet50.history['val_loss'], label='Validation Loss', color='orange')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training vs Validation Loss for Deeper ResNet-50')
plt.legend()
plt.grid(True)
plt.show()


# Training vs validation accuracy
plt.subplot(2, 2, 2)
plt.plot(history_deeper_resnet50.history['accuracy'], label='Training Accuracy', color='blue')
plt.plot(history_deeper_resnet50.history['val_accuracy'], label='Validation Accuracy', color='orange')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training vs Validation Accuracy')
plt.legend()
plt.grid(True)


plt.tight_layout()
plt.show()
